In [11]:
from __future__ import print_function
import json
import pandas as pd
from ezprettyjson import prettyjson
from ml.logutils import loadall
from ml.processing import *
from deep_dict_utils import *
import numpy as np
#%matplotlib inline
import matplotlib.pyplot as mpp
import seaborn as sns

In [12]:
with open('catalog-merged.json', 'r') as f:
    catj = json.load(f)

In [28]:
print(filter(lambda k: not 'm-' in k and not 'e-' in k, catj.keys()))
prettyjson(dmap(get('settings'), only(['runner-test-adne','runner-test-rpad','runner-test-rpag','runner-test-adad'])(catj)))

[u'runner-test-dr.50-l2e_4', u'runner-test-dr.50-l2e_3', u'ed-latest', u'final-test-off', u'ed-best', u'runner-test-dr.05-l2e_4', u'runner-test-adne', u'runner-test-dr.10-l2e_3', u'runner-test-dr.10-l2e_4', u'final-test-batch50', u'runner-test-quick', u'final-test-on', u'final-test-short', u'final-test-eg_fd-l2', u'best', u'runner-test-dr.00-l2e_3', u'runner-test-dr.00-l2e_4', u'runner-test-rpad', u'runner-test-rpag', u'latest', u'runner-test-dr.05-l2e_3', u'final-test-eg_fd-nol2', u'runner-test-adad', u'final-test-long', u'runner-test']


In [15]:
def dffd(dct):
    return pd.DataFrame.from_dict(dct)

In [32]:
data = map(pd.Series, 
           detvalues(dmap(chain(drop(['cross_validation'
                                     ,'settings']),
                          lambda d: reduce(smart_combine,
                                           detvalues(d)),
                          get('test_accuracy')),
                     only(['runner-test-adne'
                          ,'runner-test-rpad'
                          ,'runner-test-rpag'
                          ,'runner-test-adad'])(catj))))
print(map(len, data))
print(map(np.mean,data))
print(map(np.std,data))
sns.boxplot(data = data)
sns.plt.xticks([0,1,2,3],['ADAD', 'ADNE', 'RPAD', 'RPAG'])
sns.plt.xlabel('Experiment')
sns.plt.ylabel('Testnauwkeurigheid')
sns.plt.title('Vroege testen voor effect van trainingsalgoritme op prestatie')
sns.plt.show()

[17, 17, 18, 17]
[0.23391812865497075, 0.22291021671826627, 0.23326835607537363, 0.23770209838321296]
[0.064934435302356611, 0.049576197868053375, 0.054332800416807985, 0.043204191905940853]


In [73]:
only_models = {k[2:]:v for k,v in catj.iteritems() if k.startswith('m-runner-test-dr.05')}
only_models = drop(['best','latest'])(only_models)
#print(dmap(chain(only(['gaussian_base_sigma', 'test_accuracy']), 
#                 combine(['gaussian_base_sigma'], iden, 'Sigma'),
#                 keymap(lambda k: k.replace('test_accuracy', 'Testnauwkeurigheid'))), only_models))
data = dffd(reduce(smart_combine, detvalues(dmap(chain(only(['gaussian_base_sigma', 'test_accuracy']),
                                                       combine(['gaussian_base_sigma'], iden, 'Sigma'),
                                                       keymap(lambda k: k.replace('test_accuracy', 'Testnauwkeurigheid'))), 
                                                 only_models))))
data['phoney'] = range(len(data))
sns.boxplot(data = data.pivot('phoney','Sigma','Testnauwkeurigheid'))
sns.plt.ylabel('Testnauwkeurigheid')
sns.plt.title('Effect van sigma van de ruis op nauwkeurigheid in vroege experimenten')
sns.plt.show()

In [71]:
prettyjson(only_models)

In [76]:
with open('ml/optimizer.json', 'r') as f:
    cat_opt = json.load(f)
    cat_opt = drop(['e-best','e-latest','m-best','m-latest'])(cat_opt)

In [77]:
prettyjson(cat_opt)

In [81]:
# Split into categories
opt_eds = {}
opt_mods = {}
opt_res = {}
for k, v in cat_opt.iteritems():
    if k.startswith('e-'):
        opt_eds[k[2:]] = v
    elif k.startswith('m-'):
        opt_mods[k[2:]] = v
    else:
        opt_res[k] = v
# Per experiment
experiments = list(sorted(opt_res.iterkeys()))
pe_oeds, pe_omods = map(lambda x: {e: {'-'.join(k.split('-')[-2:]): v 
                                     for k,v in x.iteritems() if k.startswith(e+'-x')}
                                 for e in experiments},
                      [opt_eds, opt_mods])
# Per person
# First make a version where the x_i get replaced by the actual names, otherwise confusion occurs
name_mapping = {'-'.join(spl[:-1])+'-': '-'.join(spl[:-2])+'-'+opt_res['-'.join(spl[:-2])]['cross_validation']['drops'][int(spl[-2][1:])]+'-'
                for k in list(sorted(opt_eds.iterkeys()))
                for spl in (k.split('-'),)}
pp_oeds = keymap(lambda k: [k.replace(nmk, nmv) 
                           for nmk, nmv in name_mapping.iteritems()
                           if k.startswith(nmk)][0], 
                opt_eds)
name_mapping = {'-'.join(spl[:-1])+'-': '-'.join(spl[:-2])+'-'+opt_res['-'.join(spl[:-2])]['cross_validation']['drops'][int(spl[-2][1:])]+'-'
                for k in list(sorted(opt_mods.iterkeys()))
                for spl in (k.split('-'),)}
pp_omods = keymap(lambda k: [k.replace(nmk, nmv) 
                            for nmk, nmv in name_mapping.iteritems()
                            if k.startswith(nmk)][0], 
                 opt_mods)
pp_oeds, pp_omods = map(lambda x: {n: {'-'.join(k.split('-')[2:-2]): v
                                     for k,v in x.iteritems() if k.split('-')[-2] == n}
                                 for n in ["wannes","lawrence","zerzerzer","Wassie","Jeroen","joschout","cristaline","els","noottein"]},
                      [pp_oeds, pp_omods])
prettyjson(pe_omods)

In [110]:
# Split up different settings for omods into "experiments"
setting_strs = set(k.split('-')[-1] for k in pe_omods['final-test-optimizer'].iterkeys())
per_setting = {k: chain(combine([mk for mk in opt_mods.iterkeys() if k in mk],
                          lambda *i: reduce(smart_combine, i),
                          k),
                        get(k))(opt_mods)
               for k in setting_strs}
relevant = dmap(chain(only(['class_optimizer','test_accuracy'])), per_setting)
for key in relevant:
    relevant[key]['encdec_optimizer'] = [['rmsprop','adagrad','adadelta'][int(key[-7])] for _ in xrange(11)]
relevant = dmap(keymap(lambda k: k.replace('encdec_optimizer','AE algoritme').replace('class_optimizer','C algoritme').replace('test_accuracy','Testnauwkeurigheid')), relevant)
data = dffd(reduce(smart_combine, detvalues(relevant)))
sns.factorplot(data = data, 
               col = 'C algoritme', 
               x = 'AE algoritme', 
               y = 'Testnauwkeurigheid',
               kind = 'box')
sns.plt.suptitle('Vergelijking van combinaties van trainingsalgoritmes voor (pre-)training')
sns.plt.tight_layout()
sns.plt.show()

In [112]:
with open('ml/short-ratevar.json', 'r') as f:
    cat_rv = json.load(f)
    cat_rv = drop(['e-best','e-latest','m-best','m-latest'])(cat_rv)
prettyjson(cat_rv)

In [121]:
print(np.mean(detvalues(dmap(get('test_accuracy'),drop(['cross_validation','settings'])(cat_rv['final-test-short-ratevar'])))))
results = dmap(get('test_accuracy'),drop(['cross_validation','settings'])(cat_rv['final-test-short-ratevar']))
psc = reduce(lambda d,k: combine([rk for rk in results.iterkeys() if k in rk], 
                                 lambda *i: reduce(smart_combine,i),
                                 k)(d),
             set(k.split('-')[-1] for k in results.iterkeys()),
             results)
print(np.mean(psc['0110000']))
sns.boxplot(data = dffd(psc))
sns.plt.show()

0.538797379127
0.573999039196
